# Web Scraping with Python: An Introductory Tutorial

#### Rob Osterburg 
##### Software Engineer / Instructor

## Acknowledgements

* The generous [sponsors](http://denverdatascienceday.com/index.php/sponsors/) of [Denver Data Science Day 2017](http://denverdatascienceday.com/)

* [Galvanize](https://www.galvanize.com/pick-a-location?page=%2F) for hosting [Denver Data Science Day 2017](http://denverdatascienceday.com/)

* Bob Mickus, Tyler B. and all the volunteers from [PyData Denver](https://www.meetup.com/PyData-Denver/) who organized [Denver Data Science Day](http://denverdatascienceday.com/)

* Miguel Grinberg whose [Easy Web Scraping with Python](https://blog.miguelgrinberg.com/post/easy-web-scraping-with-python) blog post inspired this tutorial.  Miguel's posted his tutorial in 2014 and PyVideo.org has recently undergone a significant revision.

## Topics
* Which Python packages to use
* Cover key concepts, tools and techniques
* Scrape some data 
    - Organize it using a namedtuple
    - Persist it as a JSON file
    - Read it back into a namedtuple and demonstrate semantic equivilence 
* Share recommended resources

## Key Packages

* requests -- Issues HTTP requests to web servers and handles the response 

* beautifulsoup4 -- Parse the returned web page and makes it searchable

* These packages are *not* in Python's standard library, here is how to install them:
 
    - Standard Python: `pip install requests beautifulsoup4`

    - Anaconda Python: `conda install requests beautifulsoup4`


In [ ]:
# Requests is 'HTTP for Humans'
import requests

# BeautifulSoup parses and builds tree from HTML
from bs4 import BeautifulSoup

## Warning! Warning!
* Only scrape as a last resort, first see if the site has an API or other means of accessing their data
* Web scraping is commonly frowned upon by the site's owners
* **Always check the site's Terms of service, Conditions of use, and robots.txt file**   
* Aggressive scrapers can take a site down, owner's can rightfully consider that a denial of service attack
* If in doubt, talk to a lawyer, especially for *anything* work related
* Watch [Sustainable Scrapers, PyData DC, 2016](http://pyvideo.org/pydata-dc-2016/sustainable-scrapers.html) to understand how a data journalist deals with these issues. It's a good talk and quite entertaining -- in a geeky way

## PyVideo.org -- Is it OK to scrape?
* [PyVideo.org](http://pyvideo.org) provides valuable meta-data about Python presentations
* It makes finding talks easy and can be searched by event, presenter, or topic 
* But what restrictions does it have against scraping?  
* Let's take a look

## PyVideo.org - Terms and Conditions?

None that I can see, the site's [About page](http://pyvideo.org/pages/about.html) says:

> ... PyVideo.org is a freely available index of freely available resources that seek to provide everyone with the opportunity to learn about Python.

## PyVideo.org - Robots.txt?
* A robots.txt files contain *restrictions* on the content that web crawlers are permitted to access 

* The [http://pyvideo.org/robots.txt](http://pyvideo.org/robots.txt) is empty, except for one comment 

* It appears that there are no restrictions on scraping PyVideo.org  

* Let's minimize our impact on the site 

* To see *The Robots Exclusion Protocol* visit [http://www.robotstxt.org/](http://www.robotstxt.org/).

In [ ]:
# Getting the Page
resp = requests.get('http://pyvideo.org/tags.html')

# A status code other than 200 indicates problem of some sort
print('status code:', resp.status_code)

# A better approach is let requests throw an exception when a problem occurs
resp.raise_for_status()

## HTTP Responses

HTTP Verb | Effect | Success | Failure
--------- | ------ | ------- | --------
POST      | Create | 200     | 400, 40X, 500
**GET**       | **Read**   | **200**     | **400, 40X, 500**
PUT       | Update | 200     | 400, 40X, 500 
DELETE    | Delete | 200     | 400, 40X, 500

* [Comprehensive list of HTTP status codes](https://httpstatuses.com/)

In [ ]:
# Parse the Page
data = resp.text
soup = BeautifulSoup(data, 'lxml')

 ## BeautifulSoup - Parses the page
 
* Models the page as a tree
* Similar to the Document Object Model
 ![tags page structure](./images/htmltree.png)

* Parsers build the tree, pick one
 - html.parser -> Default parser for Python 3
 - HTMLParser  -> Default parser for Python 2
 - lxml        -> Fast requires installing a C library and a PyPI package 
 - html5lib    -> Pure Python and part of the standard library
 
 cite: https://interactivepython.org/runestone/static/pythonds/Trees/ExamplesofTrees.html

## BeautifulSoup - Well documented and easy to use API
* [select](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors)('css selector') --> [List of Tags]

* [find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all)(tags, keyword_args, attrs={'attr', 'value'})  --> [List of Tags]
    
* [find](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find)(tags, keyword_args, attrs={'attr', 'value'}) --> Tag

* [BeautifulSoup 4 Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## Understanding Page Structure

* To extract data from a page, you need to understand its structure
* BeautifulSoup holds the tree in memory
* Look into it using the our browser's built-in developer tools
    - Mac -- Cmd + Opt + i
    - Linux -- Ctrl + Shift + i
    - Windows -- Ctrl + Shift + i

Visiting the [Tags page](http://pyvideo.org/tags) you can see the structure of the page  
![tags page structure](./images/structure-of-tags-page.png)

## Developer Tools - Getting a CSS selector
* Use the element inspector to locate items you want to scrape
* Right-click on the element > Copy > Copy CSS Selector
* Paste the selector into your code
* Make it more general by removing aspects that specific to *a* particular tag
* In this case, `nth-child()` and `.col-md-3`
* The idea here is to tune the selector to the data you want to scrape
* Tutorial: [An Intro to CSS: Finding CSS Selectors](https://dailypost.wordpress.com/2013/07/25/css-selectors/)

In [ ]:
# Build a list of anchor elements using a CSS selector
# First we need to clean our CSS using a for-loop
topics = []
as_copied_css = '#element-list > li:nth-child(1361) > a'
cleaned_css = 'li > a'
for topic in soup.select(cleaned_css):
    topics.append(topic)
topics[:8]

## Why List Comprehensions?
* List comprehensions are a powerful way to create lists
* For-loop code above == List comprehension below
* The CSS selector got us 90% of the way there
* List comprehension can do the rest
* I find them very handy, I think you will too
* Tutorial: [Understanding List Comprehensions in Python 3](https://www.digitalocean.com/community/tutorials/understanding-list-comprehensions-in-python-3)

In [ ]:
# Now let's using a list comprehension 
# to build the list of links
topics = [topic                                        # adds the topic to the list
          for topic in soup.select('li > a')]          # loop over Tags from select
topics[:8]

* Now lets look at one of the elements in our list
* Each element is a BeautifulSoup Tag object
* *Tag* allows access to all the element's data including
    - *attributes* that are accessed using dictionary like syntax
    - *contents* that are displayed on the page

In [ ]:
# Let's look a Tag instance
topic = topics[7]
# Access the data it holds
type(topic), topic, topic.contents, topic['href']

* List comprehensions are concise and powerful to process data
* From the list of anchor elements
* Let's create a list of links to topics pages
* Filtering out all unrelated links

In [ ]:
# Extracting and filtering the links
topics = [topic['href']                       # extract attribute value
          for topic in soup.select('li > a')  # loop over anchors
          if '/tag/' in topic['href']]        # filter
         
len(topics), type(topics[0]), topics[:5]

In [ ]:
# Get the page
resp = requests.get('http://pyvideo.org/tag/scraping')
resp.status_code

In [ ]:
# Parse the page
data = resp.text
soup = BeautifulSoup(data, 'lxml')

## Using Tag.find_all()
* We have used `select()`, not let's try out `find_all()` 
* Our goal is create a list of the links to all the talks on a particular topic

In [ ]:
# Find all the video page links using find_all()
h4_tags = soup.find_all('h4', class_='entry-title')
h4_tags[0]

## BeautifulSoup in action
* Methods: `find_all()` and `select()` both return list of Tag objects
* Accessing descendents:  Tag.Descendent_Tag['attribute']

In [ ]:
# Extracting the link from <a> tag inside the <h4>
findall_talks = [tag.a['href'] for tag in h4_tags]
len(findall_talks), findall_talks[0]

In [ ]:
# Same thing using select() -- In this case select seems cleaner
site_url = 'http://pyvideo.org'
select_talks = [site_url + tag['href'] 
                for tag in soup.select('article > section > h4 > a')]
len(select_talks), select_talks[0]

## Moving on to extract presentation meta data
* Our ultimate destination is now in sight -- The video's presentation page
* It's the talk page -- that's where the best data are found

In [ ]:
# Get a page and parse it 
resp = requests.get(select_talks[0])
resp.raise_for_status()
soup = BeautifulSoup(resp.text, 'html.parser')
soup.select('.entry-title > a')

In [ ]:
# Extract the talk's title
titles = [title.contents[0].strip() 
          for title in soup.select('.entry-title > a')]
title = titles[0]
title

In [ ]:
soup.select('.url')

In [ ]:
# Extract the speaker's name
names = [elem.contents[0] 
         for elem in soup.select('.url')]
names

In [ ]:
# Extract the talk details
details = [detail 
           for detail in soup.select('.details-content > ul > li > a')]
details

In [ ]:
# Extract the YouTube link
links = [detail['href'] 
         for detail in details 
         if 'youtube.com' in detail['href']]
link = links[0]
link

In [ ]:
# Extract the subject tags
tags = [link.contents[0].lower() 
        for link in details 
        if 'tag' in link['href']]
tags

In [ ]:
# Extract the description
paragraphs = soup.select('.entry-content > p')
description = '\n\n'.join([p.contents[0] for p in paragraphs])
print(description)

##  Saving the Data

* Use namedtuple to capture information about PyVideo talk

* What do we need to capture
    - Talk title (string)
    - Name of presenter(s) (list)
    - Description (string)
    - Tags (list)
    - Link to video (string)
    
* Let's use JSON 
    - Awesome for persisting structured data 
    - Excellent data exchange format
    

## Why namedtuples?
* namedtuples are a lightweight container for data
* Using a name to access data elements is their advantage
* Numeric indexing is all that regular tuple provides

In [ ]:
# Storing related data in a namedtuple
from collections import namedtuple

# create the structure
pyvideo = namedtuple('pyvideo', 'title names description tags link')

# create an instance
talk_data = pyvideo(title=title, names=names, tags=tags, link=link, description=description)

# access the data and display it as a formatted string
fmt = 'title={}\nnames={}\ntags={}\nlink={}'
print(fmt.format(talk_data.title, talk_data.names, talk_data.tags, talk_data.link))

## Why JSON?
* Handles structured data well
* Excellent data exchange format
* Python supports round-tripping between namedtuples and JSON
    - namedtuple objects can be saved a JSON formatted text
    - JSON formatted text can be convert back to namedtuple objects
    - Of course, text can be written to file 

In [ ]:
# Writing the namedtuple to disk as JSON
import json

# Write to file
with open('a_video.json', 'w') as fout:
    # Note: preserve keys by saving a dictionary, not a list
    json.dump(talk_data._asdict(), fout)
    
# Read from file
with open('a_video.json', 'r') as fin:
    restored_talk_data = json.load(fin)

# Deserialize -- Note: pass as keyword arguments using **
restored_talk_data = pyvideo(**restored_talk_data)

# Compare semantically
talk_data == restored_talk_data
type(talk_data), type(restored_talk_data)

## Writing web scraping code has advantages

* Excellent source of valuable data 

* The code is approachable

## ... and presents some problems too

* Your code breaks when the site is updated

* Read the terms and conditions, otherwise ...

* Getting blocked, banned, sued ...

## Resources
1. [Automate the Boring Stuff](https://automatetheboringstuff.com/), [Chapter 11 — Web Scraping](https://automatetheboringstuff.com/chapter11/) by Al Sweigart (Free PDF version online).  Takes you through topics step-by-step, includes using Selenium to fill out forms and simulate mouse clicks. 

1. [RealPython Blog -- Web Scraping With Scrapy and MongoDB](https://realpython.com/blog/python/web-scraping-with-scrapy-and-mongodb/) by Micheal Herman. Scrapy is a Python package that makes scraping code easier to maintain. 

1. [Talk Python to Me Podcast -- Web scraping at scale with Scrapy and ScrapingHub](https://talkpython.fm/episodes/show/50/web-scraping-at-scale-with-scrapy-and-scrapinghub). Web scraping as a Service from the author of Scrapy.

1. [PyVideo.org](http://pyvideo.org/)— Comprehensive catalog of videos of over 8000 of Python related presentations. Talks on scraping web pages can be found on the [Scraping page](http://pyvideo.org/tag/scraping/). 

1. [Web Scraping with Python: Collecting Data from the Modern Web](https://www.amazon.com/Web-Scraping-Python-Collecting-Modern/dp/1491910291) by Ryan Mitchell.  This 4.5 star book on Amazon covers scraping topics in depth.

1. [Awesome Python](https://awesome-python.com/) -- PyPI has over 100,000 packages.  Awesome Python is a curated list of the best, see their recommended web scraping packages [here](https://awesome-python.com/#web-crawling).

1. Other Practice Sites
    * [Books to Scrape](http://books.toscrape.com/)
    * [Quotes to Scrape](http://quotes.toscrape.com/) 

In [ ]:
'''
Goes to Wikipedia to get ticker symbols, and CIK codes for S&P 500 companies
'''

url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data, 'lxml')
company_table = soup.find_all('tr')[1:506]
company_dict = dict()

for row in company_table:
    row = row.find_all('td')
    # slice into table to grab ticker and CIK
    company_dict[row[0].get_text()] = row[7].get_text()

In [ ]:
company_dict

In [ ]:
# https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=0001018724&type=8-k&dateb=20180101&owner=exclude&count=10

ticker = 'AMZN'
cik, priorto, count = company_dict[ticker], 20180101, 10
base_url = 'http://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK='\
    +str(cik)+'&type=8-K&dateb='\
    +str(priorto)+'&owner=exclude&output=xml&count='\
    +str(count)
base_url

In [ ]:
r = requests.get(base_url)
data = r.text
soup = BeautifulSoup(data, 'lxml')

In [ ]:
# Get the URL for each 8-K detail page using the 'filinghref' html tag:
detail_urls = soup.find_all('filinghref')
detail_urls

In [ ]:
forms = []

# Iterate through each URL:
for url in detail_urls:
    # If URL suffix is 'htm', change to 'html'
    url = url.string
    if url.split('.')[len(url.string.split('.')) - 1] == 'htm':
        url+='l'
            
    # Turn each URL into soup
    r = requests.get(url)
    data = r.text
    soup = BeautifulSoup(data, 'lxml')
    
    # Locate the filing date from the detail page by searching by html tag
    filing_date = soup.find_all('div', {"class" : 'info'})[1].string
    
    # The suffix to the form 8-K can be found on this page too
    url_suffix = soup.find_all('tr')[1].find_all('td')[2].get_text()
    
    # Slice off the end of our url, and add the new suffix to get the Form 8-K url
    doc_url = url[0:-31] + url_suffix
    
    # Turn the Form 8-k url into soup
    r = requests.get(doc_url)
    data = r.text
    soup = BeautifulSoup(data, 'lxml')
    
    # Finally, the text we're after! 
    # Get rid of the commas, quotation marks, and any new lines characters
    text = soup.get_text().replace(',', '').replace('"', '').replace('\n', ' ')
    
    # Combine the data into a CSV style string and write to the file created before the for loop started
    write_string = ticker + ',' + filing_date + ',' + text + '\n'
    
    forms.append(write_string)

In [ ]:
forms[0]

In [ ]:
r = requests.get(detail_urls[0].string)
data = r.text
soup = BeautifulSoup(data, 'lxml')
filing_date = soup.find_all('div', {"class" : 'info'})[1].string

In [ ]:
filing_date